In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import math
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import joblib
import scipy.sparse
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import warnings; warnings.simplefilter('ignore')
import gzip

def parse(path):
  g = gzip.open(path, 'rt', encoding='utf-8')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

review_data = getDF('Data/meta_AMAZON_FASHION.json.gz')
data_set = review_data[:30000]
data_set

,title,image,brand,feature,rank,date,asin,description,price,also_view,also_buy,fit,details,similar_item,tech1
0,Slime Time Fall Fest [With CDROM and Collector...,[https://images-na.ssl-images-amazon.com/image...,Group Publishing (CO),[Product Dimensions:\n \n8....,"13,052,976inClothing,Shoesamp;Jewelry(",8.70 inches,0764443682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,XCC Qi promise new spider snake preparing men'...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,"11,654,581inClothing,Shoesamp;Jewelry(",5 star,1291691480,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Magical Things I Really Do Do Too!,[https://images-na.ssl-images-amazon.com/image...,Christopher Manos,[Package Dimensions:\n \n8....,"19,308,073inClothing,ShoesJewelry(",5 star,1940280001,[For the professional or amateur magician. Ro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Ashes to Ashes, Oranges to Oranges",[https://images-na.ssl-images-amazon.com/image...,Flickerlamp Publishing,[Package Dimensions:\n \n8....,"19,734,184inClothing,ShoesJewelry(",5 star,1940735033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aether & Empire #1 - 2016 First Printing Comic...,[https://images-na.ssl-images-amazon.com/image...,NaN,[Package Dimensions:\n \n10...,"10,558,646inClothing,Shoesamp;Jewelry(",5 star,1940967805,NaN,$4.50,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29997,Honeystore Women's Cap Sleeve Crystal Evening ...,[https://images-na.ssl-images-amazon.com/image...,Honeystore,[Package Dimensions:\n \n15...,"22,005,842inClothing,ShoesJewelry(",5 star,B00F5V4JJ2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29998,Honeystore Women's Cap Sleeve Crystal Evening ...,[https://images-na.ssl-images-amazon.com/image...,Honeystore,[Package Dimensions:\n \n15...,"18,933,551inClothing,ShoesJewelry(",5 star5 star (0%),B00F5V4RDA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29999,AllCraft 1000x 14 Facets Resin Rhinestone Gem ...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,"5,687,062inClothing,ShoesJewelry(",jw00541,B00F5V9758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30000,Honeystore Women's Cap Sleeve Crystal Evening ...,[https://images-na.ssl-images-amazon.com/image...,Honeystore,[Package Dimensions:\n \n15...,"24,206,139inClothing,ShoesJewelry(",5 star,B00F5V4IAC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data_set.drop(['feature', 'date', 'description', 'price', 'also_view', 'also_buy', 'fit', 'details', 'similar_item', 'tech1'], axis=1,inplace=True)

In [5]:
data_set.drop(['image', 'rank'],axis=1,inplace=True)

In [6]:
data_set

,title,brand,asin
0,Slime Time Fall Fest [With CDROM and Collector...,Group Publishing (CO),0764443682
1,XCC Qi promise new spider snake preparing men'...,NaN,1291691480
2,Magical Things I Really Do Do Too!,Christopher Manos,1940280001
3,"Ashes to Ashes, Oranges to Oranges",Flickerlamp Publishing,1940735033
4,Aether & Empire #1 - 2016 First Printing Comic...,NaN,1940967805
...,...,...,...
29997,Honeystore Women's Cap Sleeve Crystal Evening ...,Honeystore,B00F5V4JJ2
29998,Honeystore Women's Cap Sleeve Crystal Evening ...,Honeystore,B00F5V4RDA
29999,AllCraft 1000x 14 Facets Resin Rhinestone Gem ...,NaN,B00F5V9758
30000,Honeystore Women's Cap Sleeve Crystal Evening ...,Honeystore,B00F5V4IAC


In [7]:
data_set.drop(['brand'], axis = 1, inplace=True)

In [8]:
data_set

,title,asin
0,Slime Time Fall Fest [With CDROM and Collector...,0764443682
1,XCC Qi promise new spider snake preparing men'...,1291691480
2,Magical Things I Really Do Do Too!,1940280001
3,"Ashes to Ashes, Oranges to Oranges",1940735033
4,Aether & Empire #1 - 2016 First Printing Comic...,1940967805
...,...,...
29997,Honeystore Women's Cap Sleeve Crystal Evening ...,B00F5V4JJ2
29998,Honeystore Women's Cap Sleeve Crystal Evening ...,B00F5V4RDA
29999,AllCraft 1000x 14 Facets Resin Rhinestone Gem ...,B00F5V9758
30000,Honeystore Women's Cap Sleeve Crystal Evening ...,B00F5V4IAC


In [11]:
title = data_set.rename(columns={'asin':'Item'})

In [12]:
title

,title,Item
0,Slime Time Fall Fest [With CDROM and Collector...,0764443682
1,XCC Qi promise new spider snake preparing men'...,1291691480
2,Magical Things I Really Do Do Too!,1940280001
3,"Ashes to Ashes, Oranges to Oranges",1940735033
4,Aether & Empire #1 - 2016 First Printing Comic...,1940967805
...,...,...
29997,Honeystore Women's Cap Sleeve Crystal Evening ...,B00F5V4JJ2
29998,Honeystore Women's Cap Sleeve Crystal Evening ...,B00F5V4RDA
29999,AllCraft 1000x 14 Facets Resin Rhinestone Gem ...,B00F5V9758
30000,Honeystore Women's Cap Sleeve Crystal Evening ...,B00F5V4IAC


SyntaxError: 'return' outside function (<ipython-input-13-d08e57991806>, line 1)